In [1]:
import pandas as pd 
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup

검색어, 저장 할 파일명

In [2]:
keyword = '강제동원조사법'

In [3]:
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service('chromedriver.exe'), options=chrome_options)

driver.get("https://www.bigkinds.or.kr/v2/news/index.do")
time.sleep(1)

# 검색창에 '검색어' 검색
element = driver.find_element(By.ID, "total-search-key")
element.send_keys(keyword)
time.sleep(1)

# '기간' 클릭
driver.find_element(By.XPATH, '//*[@id="collapse-step-1"]').click()
driver.find_element(By.XPATH,'//*[@id="collapse-step-1"]').click()
time.sleep(1)
driver.find_element(By.XPATH,'//*[@id="collapse-step-1-body"]/div[3]/div/div[1]/div[1]/a').click()
time.sleep(1)

# 날짜 입력
driver.find_element(By.XPATH, '//*[@id="search-begin-date"]').send_keys(Keys.CONTROL + "a");
driver.find_element(By.XPATH, '//*[@id="search-begin-date"]').send_keys(Keys.DELETE);
driver.find_element(By.XPATH, '//*[@id="search-begin-date"]').send_keys('2011-11-01')

# 적용버튼 클릭
driver.find_element(By.XPATH, '//*[@id="search-foot-div"]/div[2]/button[2]').click()
time.sleep(3)

# 100건씩 보기
Select(driver.find_element(By.ID, 'select2')).select_by_value('100')
time.sleep(3)

# 총 개수 확인
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
n = soup.find('span', 'total-news-cnt').text
n = int(n)
print(n)

10


In [4]:
# 본문내용 담을 리스트 생성
conts_li = []

# 본문내용 담기
for i in range(n):
    try:
        # 기사제목 클릭
        driver.find_element(By.XPATH,'//*[@id="news-results"]/div[{}]/div/div[2]/a'.format(i+1)).click()
        time.sleep(1)
        cont = driver.find_element(By.XPATH,'//*[@id="news-detail-modal"]/div/div/div[1]/div/div[2]')
        contents = cont.text
        contents = contents.replace('\n','').replace('\'','')
        conts_li.append(contents)

        # 창 끄기
        driver.find_element(By.XPATH,'//*[@id="news-detail-modal"]/div/div/button').click()
        time.sleep(1)
    except:
        conts_li.append('None')

## 기존 본문내용 잘렸던 파일 불러오기
* 기존 csv파일에 새로 만든 csv파일 붙여넣기 안됨 주의!!

In [ ]:
# 엑셀파일에 붙여넣기
df = pd.read_csv('{}.csv'.format(keyword))
df.head()

In [ ]:
# 엑셀파일 개수랑 맞는지 확인!!!!
print(len(df))
print(len(conts_li))

In [7]:
# 본문 내용 넣고 csv 파일로 저장
df['본문'] = conts_li
df.to_csv('./{}.csv'.format(keyword), index=False, encoding='utf-8-sig')